### Vector stores and retrievers
This video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover 
- Documents
- Vector stores
- Retrievers

### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:

from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROG_APIKEY_RETRIEVER')

os.environ['HUGGINGFACE_API_KEY'] = os.getenv("HUGGINGFACE_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model="Qwen-Qwq-32b")
llm




ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000168035C2180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000168035C35C0>, model_name='Qwen-Qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
#embeddings
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embedding

c:\Users\user\Desktop\yeni layiheler\langchain(lesson udemy(Gen AI 3)\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
#Vectorstore
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding=embedding)

vectorstore

In [10]:
vectorstore.similarity_search("")

[Document(id='45854429-0616-4c03-84d3-18e94f0e77ea', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='ef15b8fb-132e-4fbc-b514-6a940f547396', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='1c8aab2f-fc1c-49b7-828e-f44bd56ef54b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='99a94c76-0f3f-498d-a8da-51ecb085bd6d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [25]:
#async query
await vectorstore.asimilarity_search("cat")

[Document(id='1c8aab2f-fc1c-49b7-828e-f44bd56ef54b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='ef15b8fb-132e-4fbc-b514-6a940f547396', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='99a94c76-0f3f-498d-a8da-51ecb085bd6d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='45854429-0616-4c03-84d3-18e94f0e77ea', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [27]:
vectorstore.similarity_search_with_score('cat')

[(Document(id='1c8aab2f-fc1c-49b7-828e-f44bd56ef54b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058006286621),
 (Document(id='ef15b8fb-132e-4fbc-b514-6a940f547396', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='99a94c76-0f3f-498d-a8da-51ecb085bd6d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956904888153076),
 (Document(id='45854429-0616-4c03-84d3-18e94f0e77ea', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.66579270362854)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [ ]:
git initgit initgit initgit initgit init

In [ ]:
##Retrievers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat','dog'])

[[Document(id='1c8aab2f-fc1c-49b7-828e-f44bd56ef54b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ef15b8fb-132e-4fbc-b514-6a940f547396', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [38]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(['cat','dog'])

[[Document(id='1c8aab2f-fc1c-49b7-828e-f44bd56ef54b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ef15b8fb-132e-4fbc-b514-6a940f547396', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
#RAG


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
            Answer this questions using the provided context only
            
        {question}

        Context:
        {context}
        """

prompt = ChatPromptTemplate.from_messages([('human',message)])

rag_chain = {"context":retriever,'question':RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about cat")
print(response.content)



<think>
Okay, the user is asking me to "tell me about cat" and I need to use the provided context. Let me check the context given.

The context has one document with the ID '1c8aab2f-fc1c-49b7-828e-f44bd56ef54b' from the source 'mammal-pets-doc'. The page_content says, "Cats are independent pets that often enjoy their own space." 

So, I should base my answer solely on that sentence. The user's question is general, so I need to summarize what the context provides. The key points from the context are that cats are pets, they are independent, and they like their own space. 

I should make sure not to add any extra information beyond what's in the context. For example, I shouldn't mention things like their diet, behavior beyond independence, or physical characteristics unless they're in the provided text. 

Let me structure the answer clearly, stating that according to the source, cats are independent and prefer their own space. Keep it concise and straightforward.
</think>

According to